In [2]:
import gmsh

In [11]:
gmsh.initialize()

gmsh.model.add("Student's room")

# Параметры комнаты
room_width = 3.8
room_height = 3.1
room_length = 5.6

# Параметры окна
window_width = 1.6
window_length = 1.5
window_offset = 1.4     # отступ окна от пола

# Параметры батареи
heater_width = 1.5
heater_height = 1
heater_depth = 0.14    # толщина батареи
heater_offset = 0.2   # расстояние от пола до нижней части батареи

room = gmsh.model.occ.addBox(0, 0, 0, room_width, room_length, room_height)
x_window = (room_width - window_width) / 2
y_window = room_length
z_window = window_offset
window = gmsh.model.occ.addBox(x_window, y_window, z_window, window_width, 0.1, window_length)


x_heater = x_window + (window_width - heater_width) / 2  # центрирование батареи под окном
y_heater = y_window - heater_depth
z_heater = heater_offset
heater = gmsh.model.occ.addBox(x_heater, y_heater, z_heater, heater_width, heater_depth, heater_height)

air_volume = gmsh.model.occ.cut([(3, room)], [(3, window), (3, heater)], removeTool=True)
gmsh.model.occ.synchronize()

# Получаем точки на поверхностях радиатора для использования в поле Distance
heater_surfaces = gmsh.model.getEntitiesInBoundingBox(
    x_heater, y_heater, z_heater,
    x_heater + heater_width, y_heater + heater_depth, z_heater + heater_height, 2
)

# Получаем центральные точки поверхностей радиатора
points = []
for surface in heater_surfaces:
    com = gmsh.model.occ.getCenterOfMass(surface[0], surface[1])
    points.append(com)

# Создание поля Distance и добавление точек
distance_field = gmsh.model.mesh.field.add("Distance")
gmsh.model.mesh.field.setNumbers(distance_field, "PointsList", points)

# Настройка поля Threshold для контроля размера сетки
threshold_field = gmsh.model.mesh.field.add("Threshold")
gmsh.model.mesh.field.setNumber(threshold_field, "InField", distance_field)
gmsh.model.mesh.field.setNumber(threshold_field, "SizeMin", 0.02)  # Уменьшение размера до 0.02
gmsh.model.mesh.field.setNumber(threshold_field, "SizeMax", 0.1)   # Размер вне области радиатора
gmsh.model.mesh.field.setNumber(threshold_field, "DistMin", 0.05)  # Минимальное расстояние
gmsh.model.mesh.field.setNumber(threshold_field, "DistMax", 0.5)   # Максимальное расстояние


gmsh.model.mesh.field.setAsBackgroundMesh(threshold_field)
#gmsh.model.occ.synchronize() #может помочь исправить дублирование вершин сетки

gmsh.model.mesh.generate(3)
gmsh.write("room_with_refined_mesh_on_heater.msh2")
gmsh.write("room_with_refined_mesh_on_heater.vtk")


gmsh.finalize()